In [10]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, DateType, LongType, BooleanType, FloatType, TimestampType
from pyspark.sql import Row 

In [2]:
spark = (SparkSession.builder
    .appName("example")
    .config("spark.sql.catalog.mycatalog", "org.apache.iceberg.spark.SparkCatalog")
    # .config("spark.sql.catalog.mycatalog.type", "rest")
    # .config("spark.sql.catalog.mycatalog.uri", "http://localhost")
    .config("spark.sql.catalog.mycatalog.type", "hive")
    .config("spark.sql.catalog.mycatalog.uri", "thrift://hive-metastore")
    .getOrCreate()
)

25/11/20 02:06:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark.sql("select * from demo.nyc.company")

DataFrame[id: bigint, name: string, test: boolean, employee: array<struct<id:bigint,name:string,contract:array<struct<id:bigint,contract_id:string,since:date,until:date,position:string>>>>]

In [4]:
company_schema = StructType([
    StructField("id", LongType(), nullable=False),
    StructField("name", StringType(), nullable=False),
    StructField("test", BooleanType(), nullable=False),
    StructField("employee", ArrayType(
        StructType([
            StructField("id", LongType(), nullable=False),
            StructField("name", StringType(), nullable=False),
            StructField("contract", ArrayType(
                StructType([
                    StructField("id", LongType(), nullable=False),
                    StructField("contract_id", StringType(), nullable=False),
                    StructField("since", DateType(), nullable=False),
                    StructField("until", DateType(), nullable=True),
                    StructField("position", StringType(), nullable=True)
                ])
            ), nullable=True)
        ])
    ), nullable=True),
])

In [5]:
spark.sql("""
create table if not exists demo.nyc.company (
    id bigint not null,
    name string not null,
    test boolean not null,
    employee array<
                struct<
                    id bigint not null,
                    name string not null,
                    contract array<
                        struct<
                            id bigint not null,
                            contract_id string not null,
                            since date not null,
                            until date,
                            position string
                        >
                    >
                >
            >
)
using iceberg
partitioned by (
    id
)
"""
)

DataFrame[]

In [41]:
payslip_schema = StructType([
    StructField("id", LongType(), nullable=False),
    StructField("type", StringType(), nullable=False),
    StructField("contract_id", LongType(), nullable=False),
    StructField("company_id", LongType(), nullable=False),
    StructField("period", DateType(), nullable=False),
    StructField("created_at", DateType(), nullable=False),
    StructField("employee_id", LongType(), nullable=False),
    StructField("item", ArrayType(
        StructType([
            StructField("id", LongType(), nullable=False),
            StructField("name", StringType(), nullable=False),
            StructField("value", FloatType(), nullable=False)
        ])
    ), nullable=False)
])

In [8]:
sql_payslip = """
create table if not exists demo.nyc.payslip (
    id bigint not null,
    type string not null,
    contract_id bigint not null,
    company_id bigint not null,
    period date not null,
    created_at date not null,
    employee_id bigint not null,
    item array<
            struct<
                id bigint not null,
                name string not null,
                value float not null
            >
        >
)
using iceberg
partitioned by (
    company_id,
    year(period)
)
"""
spark.sql(sql_payslip)

DataFrame[]

In [50]:
workshift_schema = StructType([
    StructField("id", LongType(), nullable=False),
    StructField("type", StringType(), nullable=False),
    StructField("company_id", LongType(), nullable=False),
    StructField("period", DateType(), nullable=False),
    StructField("since", DateType(), nullable=False),
    StructField("until", DateType(), nullable=False),
    StructField("employee_id", LongType(), nullable=False),
    StructField("mark", ArrayType(
        StructType([
            StructField("id", LongType(), nullable=False),
            StructField("type", StringType(), nullable=False),
            StructField("ts", TimestampType(), nullable=False),
            StructField("status", StringType(), nullable=False)
        ])
    ), nullable=False)
])

In [12]:
sql_workshift = """
create table if not exists demo.nyc.workshift (
    id bigint not null,
    type string not null,
    company_id bigint not null,
    period date not null,
    since date not null,
    until date not null,
    employee_id bigint not null,
    mark array<
            struct<
                id bigint not null,
                type string not null,
                ts timestamp not null,
                status string not null
            >
        >
)
"""
spark.sql(sql_workshift)

DataFrame[]

In [17]:
from faker import Faker
import random
import uuid
from datetime import date
from dateutil.relativedelta import relativedelta

fake = Faker(['es_CL'])

company_data = []
employee_ids = []
employee_id, contract_id = 0, 0
for i in range(100):
    data = {}
    data["id"] = i
    data["name"] = fake.company()
    data["test"] = i < 95
    data["employee"] = []
    for ii in range(random.randint(0,500)):
        contracts = []
        for iii in range(random.randint(1,10)):
            if iii == 0:
                since = fake.date_between(start_date='-10y', end_date='-1w')
            else:
                since = until - relativedelta(days=1)
            until = since + relativedelta(days=random.randint(30,500))
            contracts.append({
                "id": iii,
                "contract_id": uuid.uuid4(),
                "since": since,
                "until": until,
                "position": fake.job()
            })
            employee_ids.append((i,employee_id,contract_id))
            contract_id += 1
        data["employee"].append({
            "id": ii,
            "name": fake.name(),
            "contract": contracts
        })
        employee_id += 1
    company_data.append(data)

In [21]:
# delete_sql = """
# truncate table demo.nyc.company
# """
# spark.sql(delete_sql)
company_df = spark.createDataFrame(company_data, company_schema)
company_df.writeTo("demo.nyc.company").append()

In [43]:
payslip_data = []
item = 0
payslip = 0
for company, employee, contract in employee_ids:
    items = []
    for _ in range(random.randint(1,10)):
        items.append({
            "id": item,
            "name": random.choice(['sueldo líquido', 'sueldo bruto', 'bono marzo', 'bono navidad', 'bono', 'salud', 'afp', 'seguro cesantía', 'adelanto', 'vacaciones']),
            "value": float(random.randint(50000,1500000))
        })
        item += 1
    payslip_data.append({
        "id": payslip,
        "type": random.choice(['sueldo', 'reproceso', 'finiquito']),
        "contract_id": contract,
        "company_id": company,
        "period": fake.date_between(start_date='-1y', end_date='-1d'),
        "created_at": fake.date_between(start_date='-1y', end_date='-1d'),
        "employee_id": employee,
        "item": items
    })
    payslip += 1

In [52]:
from datetime import datetime

workshift_data = []
workshift = 0
mark = 0
cmp, emp, cnt = zip(*employee_ids)
employee_ids_wt = list(set(zip(cmp, emp)))
for employee, company in employee_ids_wt:
    marks = []
    for _ in range(random.randint(2,60)):
        marks.append({
            "id": mark,
            "type": random.choice(['zkteco', 'mobile', 'totem']),
            "ts": fake.date_time_between(start_date="-1y", end_date="-1d"),
            "status": random.choice(['ok', 'not ok'])
        })
        mark += 1
    workshift_data.append({
        "id": workshift,
        "type": random.choice(['lunes a viernes', 'nocturno', 'rotativo']),
        "company_id": company,
        "period": fake.date_between(start_date="-1y", end_date="-1d"),
        "since": fake.date_between(start_date="-1y", end_date="-1d"),
        "until": fake.date_between(start_date="-1y", end_date="-1d"),
        "employee_id": employee,
        "mark": marks
    })
    workshift += 1

In [44]:
payslip_df = spark.createDataFrame(payslip_data, payslip_schema)
payslip_df.writeTo("demo.nyc.payslip").append()

25/11/21 00:32:53 WARN TaskSetManager: Stage 9 contains a task of very large size (1092 KiB). The maximum recommended task size is 1000 KiB.


In [53]:
workshift_df = spark.createDataFrame(workshift_data, workshift_schema)
workshift_df.writeTo("demo.nyc.workshift").append()

25/11/21 00:38:12 WARN TaskSetManager: Stage 12 contains a task of very large size (1283 KiB). The maximum recommended task size is 1000 KiB.


In [54]:
workshift_df

DataFrame[id: bigint, type: string, company_id: bigint, period: date, since: date, until: date, employee_id: bigint, mark: array<struct<id:bigint,type:string,ts:timestamp,status:string>>]